## K-means

v1.0

In [2]:
# 初始化
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题

# 获取当前运行的python代码地址
import os
path = os.getcwd()
# path

In [16]:
# 数据读取
import pandas as pd
data = pd.read_csv('data.txt',sep='\n',header=None,names=['coordination_source'])
# data

# 数据预处理
import re
pattern = re.compile(r"name:'(\w+)',\s+geoCoord:\[(\d+.\d+),\s(\d+.\d+)\]")
city_name=[]
city_gps=[]
for i in data['coordination_source']:
    city_info = pattern.findall(str(i))
    city, long, lat = city_info[0]
    city_name.append(city)
    city_gps.append(float(long),float(lat))
city_info = {'name':city_name,'gps': city_gps}
city_info

{'gps': [(103.73, 36.03),
  (98.17, 39.47),
  (101.74, 36.56),
  (104.06, 30.67),
  (114.48, 38.03),
  (102.73, 25.04),
  (106.71, 26.57),
  (114.31, 30.52),
  (113.65, 34.76),
  (117.0, 36.65),
  (118.78, 32.04),
  (117.27, 31.86),
  (120.19, 30.26),
  (115.89, 28.68),
  (119.3, 26.08),
  (113.23, 23.16),
  (113.0, 28.21),
  (110.35, 20.02),
  (123.38, 41.8),
  (125.35, 43.88),
  (126.63, 45.75),
  (112.53, 37.87),
  (108.95, 34.27),
  (121.3, 25.03),
  (116.46, 39.92),
  (121.48, 31.22),
  (106.54, 29.59),
  (117.2, 39.13),
  (111.65, 40.82),
  (108.33, 22.84),
  (91.11, 29.97),
  (106.27, 38.47),
  (87.68, 43.77),
  (114.17, 22.28),
  (113.54, 22.19)],
 'name': ['兰州',
  '嘉峪关',
  '西宁',
  '成都',
  '石家庄',
  '拉萨',
  '贵阳',
  '武汉',
  '郑州',
  '济南',
  '南京',
  '合肥',
  '杭州',
  '南昌',
  '福州',
  '广州',
  '长沙',
  '海口',
  '沈阳',
  '长春',
  '哈尔滨',
  '太原',
  '西安',
  '台湾',
  '北京',
  '上海',
  '重庆',
  '天津',
  '呼和浩特',
  '南宁',
  '西藏',
  '银川',
  '乌鲁木齐',
  '香港',
  '澳门']}

In [4]:
# 定义距离计算公式，采用Haversine距离
import math
def Haversine_Distance(pointA, pointB):
    #地球半径约为
    radius = 6371  # km
    lon1, lat1 = pointA
    lon2, lat2 = pointB
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c
    return d

In [51]:
import random
def Kmeans_Function(train_iter, k, num_epochs, stop_d):
    city_gps = train_iter['gps']
    city_long = list(zip(*city_gps)) [0]
    city_lat = list(zip(*city_gps)) [1]
    # 随机初始化 K 个簇中心
    cluster_point=[]
    for i in range(k):
        cp_x1 = random.uniform(min(city_long), max(city_long))
        cp_x2 = random.uniform(min(city_lat), max(city_lat))
        cluster_point.append((cp_x1,cp_x2))
    # 将样本点分簇
    cluster=[]
    for j in range(len(city_gps)):
        distance=[]
        for i in range(k):
            distance.append(Haversine_Distance(train_iter['feature'][j],cluster_point[i]))
        cluster.append(distance.index(max(distance, key = abs)))
